In [ ]:
import re
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from dataset import NeuralPhysDataset

def custom_sort(x):
    numbers = re.findall(r'\d+', x)
    return int(numbers[0]) if numbers else float('inf')
def show_pre(pre):
    pre_=[]
    for i in pre:
        i = i.reshape(3,128,256).detach().transpose(2,0).numpy()
        pre_.append(np.swapaxes(i[:128], 0, 1))
    return pre_ 


traj_index=892
val_dataset = NeuralPhysDataset(flag='val')
image_folder = 'training_data/'+ 'single_pendulum'+'/{}'.format(traj_index)
images = sorted(os.listdir(image_folder), key=custom_sort)
x0 = val_dataset.get_data(os.path.join(image_folder, images[0]))
x1 = val_dataset.get_data(os.path.join(image_folder, images[1]))
X_0 = torch.cat([x0, x1], 2)
def Img(frame):
    x0 = val_dataset.get_data(os.path.join(image_folder, images[frame]))
    x1 = val_dataset.get_data(os.path.join(image_folder, images[frame+1]))
    x = torch.cat([x0, x1], 2)
    img_true = x.transpose(2,0).numpy()
    
    return x, np.swapaxes(img_true[:128], 0, 1)
################## Ours
seed=0
filename='single_pendulum' + '_caehyp_vp{}'.format(seed)
cae_path='outputs/'+filename+'/model_best.pkl'
cae=torch.load(cae_path,map_location='cpu').eval()

filename = 'latent_'+'single_pendulum' + '_caehyp_vp{}'.format(seed)
latent_net_path='outputs/'+filename+'/model_best.pkl'
net=torch.load(latent_net_path,map_location='cpu').eval()

x0= cae.encoder(X_0.unsqueeze(0)).squeeze()
latent_pre = net.predict(x0, h=net.h, steps=60, keepinitx=True)
pre_cae=[]
for i in range(latent_pre.shape[0]):
    pre_cae.append(cae.decoder(latent_pre[i:i+1]))
pre_cae=show_pre(pre_cae) 


####################             PLOT          #####################
fig, ax=plt.subplots(2, 8, figsize=(12,4))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.02, hspace=0.4)


a = Img(0) 

for i in range(8): 
    traj_i=1+4*i
    x, ima_true = Img(traj_i*2)
    ax[0,i].imshow(ima_true)
    ax[0,i].axis('off')
    ax[1,i].imshow(pre_cae[traj_i*2])
    ax[1,i].axis('off')
    
    ax[1,i].text(70,150, 't={}/60'.format(traj_i*2), fontsize=18, color='black',
      horizontalalignment='center', verticalalignment='center')
   

    
titlesize=25
ax[0,0].set_title('Ground truth', fontsize=titlesize, loc='left')
ax[1,0].set_title('Prediction of CpAE', fontsize=titlesize, loc='left')
plt.show()